# LSTM baseline

from kuto

In [1]:
import os
import sys
import glob
import pickle
import random

import numpy as np
import pandas as pd
import scipy.stats as stats
from pathlib import Path

from timm.models.densenet import densenet121
from functools import partial
import glob
import albumentations as A
from albumentations.pytorch import ToTensorV2


sys.path.append('../../')
import src.utils as utils
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim

import pytorch_lightning as pl
# from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import EarlyStopping

import wandb
from pytorch_lightning.loggers import WandbLogger

from catalyst.dl import SupervisedRunner, Runner
from catalyst.core import Callback, CallbackOrder, IRunner

In [2]:
DATA_DIR = Path("/home/knikaido/work/Indoor-Location-Navigation/data/")
WIFI_DIR = DATA_DIR / 'indoorunifiedwifids'
MLFLOW_DIR = DATA_DIR / 'mlflow/mlruns'
OUTPUT_DIR = Path('./output/')
MLFLOW_DIR = DATA_DIR / 'mlflow/mlruns'
IMG_DIR = DATA_DIR / f"indoor-location-navigation-img/metadata/"

## config

In [3]:
SEED = 777
N_SPLITS = 5
SHUFFLE = True
EPOCH = 50
OUTPUT_DIR = './output/'

DATASET_CONFIG = {
    'loader': {
      'train': {
        'batch_size': 16,
        'shuffle': True,
        'num_workers': 1,
        'pin_memory': True,
        'drop_last': True,
      },
      'valid': {
        'batch_size': 2,
        'shuffle': False,
        'num_workers': 1,
        'pin_memory': True,
        'drop_last': True,
      }
    }
}

config = DATASET_CONFIG
SAVE_TEST_SUB_PATH = "sub05.csv"

SUM = 400

In [4]:
# config
# config = configs

# globals variable
SEED = 777
MAX_EPOCHS = 200
N_SPLITS = 5
DEBUG = False
# EXP_MESSAGE = config['globals']['exp_message']

EXP_NAME = 17
IS_SAVE = True

utils.set_seed(SEED)

## read data

In [5]:
train_df = pd.read_csv(WIFI_DIR / 'train_all.csv')
test_df = pd.read_csv(WIFI_DIR / 'test_all.csv')

simple_accurate_99 = pd.read_csv('../01/submission.csv')
test_df['floor'] = simple_accurate_99['floor'].values

In [6]:
sub = pd.read_csv(DATA_DIR/'indoor-location-navigation/sample_submission.csv', index_col=0)

In [7]:
meta_img_paths = sorted(glob.glob(str(IMG_DIR / f'*/*/*.npy')))
len(meta_img_paths)

139

In [39]:
img_dir_df = pd.DataFrame(meta_img_paths)[0].apply(lambda x: pd.Series(x.split("/")))
img_dir_df = img_dir_df[[8, 9]]
img_dir_df = img_dir_df.rename(columns={8:'site_id', 9:'floor'})
img_dir_df

,site_id,floor
0,5a0546857ecc773753327266,-1.0
1,5a0546857ecc773753327266,0.0
2,5a0546857ecc773753327266,1.0
3,5a0546857ecc773753327266,2.0
4,5a0546857ecc773753327266,3.0
...,...,...
134,5dc8cea7659e181adb076a3f,2.0
135,5dc8cea7659e181adb076a3f,3.0
136,5dc8cea7659e181adb076a3f,4.0
137,5dc8cea7659e181adb076a3f,5.0


In [9]:
class ColorImageDataset(Dataset):
    def __init__(self, df: pd.DataFrame, pos_transforms=None, transforms=None):
        self.df = df
        self.site_id = df['site'].values
        self.floor = df['floor'].values
        self.pos_transforms = pos_transforms
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx: int):
        img_path = IMG_DIR / f'{self.site_id[idx]}/{self.floor[idx]}/floor_image.png.npy'
        img = np.load(img_path)
        # 負例のサンプリングを行う
        while True:
            neg_idx = np.random.choice(range(len(self.df)))
            if neg_idx != idx:
                break
        neg_img_path = IMG_DIR / f'{self.site_id[neg_idx]}/{self.floor[neg_idx]}/floor_image.png.npy'
        neg_img = np.load(neg_img_path)


        anchor = self.transforms(image=img)["image"]
        pos = self.pos_transforms(image=img)["image"]
        neg = self.transforms(image=neg_img)["image"]
        return anchor, pos, neg

In [10]:
def tensor_max(x, axis=None, keepdims=False):
    if axis is None:
        axis = range(x.ndim)
    elif isinstance(axis, int):
        axis = [axis]
    else:
        axis = sorted(axis)

    for ax in axis[::-1]:
        x = x.max(dim=ax, keepdim=keepdims)[0]

    return x

def tensor_min(x, axis=None, keepdims=False):
    if axis is None:
        axis = range(x.ndim)
    elif isinstance(axis, int):
        axis = [axis]
    else:
        axis = sorted(axis)

    for ax in axis[::-1]:
        x = x.min(dim=ax, keepdim=keepdims)[0]

    return x

In [11]:
class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn_encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3)       
        )

    def forward(self, x):
        x = self.cnn_encoder(x)
        return x.mean(dim=[2, 3]) + tensor_max(x, axis=(2,3))

In [12]:
class ContrastiveLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.cos = nn.CosineSimilarity()

    def forward(self, anchor, pos, neg):
        pos_loss = 1.0 - self.cos(anchor, pos).mean(dim=0)
        neg_loss = self.cos(anchor, neg).mean(dim=0)
        return pos_loss + neg_loss

In [13]:
class ContrastRunner(Runner):
    def predict_batch(self, batch, **kwargs):
        return super().predict_batch(batch, **kwargs)

    def handle_batch(self, batch):
        anchor, pos, neg = batch[0], batch[1], batch[2]
        anchor = anchor.to(self.device)
        pos = pos.to(self.device)
        neg = neg.to(self.device)

        anchor_emb = self.model(anchor)
        pos_emb = self.model(pos)
        neg_emb = self.model(neg)

        loss = self.criterion(anchor_emb, pos_emb, neg_emb)
        self.batch_metrics.update({
            "loss": loss
        })

        self.input = batch
        if self.is_train_loader:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

In [14]:
class SchedulerCallback(Callback):
    def __init__(self):
        super().__init__(CallbackOrder.Scheduler)

    def on_loader_end(self, state: IRunner):
        lr = state.scheduler.get_last_lr()
        state.epoch_metrics["lr"] = lr[0]
        if state.is_train_loader:
            state.scheduler.step()

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [16]:
# Learner class(pytorch-lighting)
class Learner(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.criterion = ContrastiveLoss()
        self.train_losses = []
    
    def training_step(self, batch, batch_idx):
        anchor, pos, neg = batch
        anchor_emb = self.model(anchor.to(device))
        pos_emb = self.model(pos.to(device))
        neg_emb = self.model(neg.to(device))
        
        loss = self.criterion(anchor_emb, pos_emb, neg_emb)
        self.train_losses.append(loss.item())

        return loss
    
    def training_epoch_end(self, loss):
        avg_loss = sum(self.train_losses) / len(self.train_losses)
        print(f'epoch = {self.current_epoch}, loss = {avg_loss}')
    
    def configure_optimizers(self):
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "Loss/val"}

In [18]:
# kf = KFold(n_splits=5, random_state=1213, shuffle=True)



pos_transforms = A.Compose([A.Normalize(), A.Flip(p=1.0), ToTensorV2()])
transforms = A.Compose([A.Normalize(), ToTensorV2()])
trn_dataset = ColorImageDataset(img_dir_df, pos_transforms, transforms)
#     val_dataset = ColorImageDataset(val_palette, transforms)

trn_loader = DataLoader(
    trn_dataset, batch_size=4, shuffle=True, num_workers=1)
#     val_loader = torchdata.DataLoader(
#         val_dataset, batch_size=256, shuffle=False, num_workers=20)

model = CNNModel()
learner = Learner(model)
trainer = pl.Trainer(
    max_epochs=50,
    default_root_dir=OUTPUT_DIR,
    gpus=1,
    deterministic=True,
    benchmark=True,
#         precision=16,
#         progress_bar_refresh_rate=0  # vscodeの時progress barの動作が遅いので表示しない
    )
trainer.fit(learner, train_dataloader=trn_loader)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type            | Params
----------------------------------------------
0 | model     | CNNModel        | 93.2 K
1 | criterion | ContrastiveLoss | 0     
----------------------------------------------
93.2 K    Trainable params
0         Non-trainable params
93.2 K    Total params
0.373     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

epoch = 0, loss = 0.7830335868256433
epoch = 1, loss = 0.5335671969822475
epoch = 2, loss = 0.43422786430234
epoch = 3, loss = 0.37317709353353296
epoch = 4, loss = 0.3347785527259111
epoch = 5, loss = 0.2939312360027716
epoch = 6, loss = 0.2529141334930853
epoch = 7, loss = 0.22129284809343516
epoch = 8, loss = 0.19109124480377113
epoch = 9, loss = 0.1799448665816869
epoch = 10, loss = 0.15506396743093992
epoch = 11, loss = 0.13137954317831568
epoch = 12, loss = 0.1295571790005152
epoch = 13, loss = 0.12436429733904648
epoch = 14, loss = 0.11763421392511754
epoch = 15, loss = 0.11791178018985582
epoch = 16, loss = 0.1189683756814534
epoch = 17, loss = 0.12296856273083932
epoch = 18, loss = 0.12946091011110553
epoch = 19, loss = 0.12836463743820786
epoch = 20, loss = 0.12760396267027677
epoch = 21, loss = 0.12500266113209646
epoch = 22, loss = 0.11961831359528237
epoch = 23, loss = 0.11828846125863493
epoch = 24, loss = 0.11693250926690442
epoch = 25, loss = 0.11411270833072754
epoch =

1

In [19]:
model.eval()

CNNModel(
  (cnn_encoder): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  )
)

In [20]:
test_dataset = ColorImageDataset(img_dir_df, pos_transforms, transforms)
test_loader = DataLoader(
    trn_dataset, batch_size=1, shuffle=False, num_workers=1)

In [41]:
test_res = []
for data, _, _ in test_loader:
    test_res.append(model(data).to('cpu').detach().numpy().copy())

In [42]:
column = [f"meta_embedding_{i}" for i in range(128)]

In [43]:
meta_emb_df = pd.DataFrame(np.array(test_res).reshape(139, 128), columns=column)

In [45]:
meta_emb_df = pd.concat([meta_emb_df, img_dir_df], axis=1)
meta_emb_df

,meta_embedding_0,meta_embedding_1,meta_embedding_2,meta_embedding_3,meta_embedding_4,meta_embedding_5,meta_embedding_6,meta_embedding_7,meta_embedding_8,meta_embedding_9,...,meta_embedding_122,meta_embedding_123,meta_embedding_124,meta_embedding_125,meta_embedding_126,meta_embedding_127,site_id,floor,site_id,floor
0,-0.113423,0.054440,0.014797,0.118815,0.031536,0.550194,0.122259,-0.011289,-0.004517,-0.068888,...,-0.006481,-0.001067,-0.120445,8.810782,0.051331,4.612503,5a0546857ecc773753327266,-1.0,5a0546857ecc773753327266,-1.0
1,-0.370011,-0.430872,-0.336449,-0.395386,-0.346016,-0.261797,-0.302974,-0.335987,-0.354445,-0.254917,...,-0.260484,-0.188254,-0.335425,-4.474809,-0.017301,-2.094401,5a0546857ecc773753327266,0.0,5a0546857ecc773753327266,0.0
2,-0.482859,-0.357644,-0.389540,-0.508363,-0.496089,0.194938,-0.417399,-0.420956,-0.409147,-0.310212,...,-0.296334,-0.263751,-0.444299,-3.086956,-0.028302,-1.587106,5a0546857ecc773753327266,1.0,5a0546857ecc773753327266,1.0
3,-0.608664,-0.396513,-0.538601,-0.368761,-0.533122,0.013818,-0.407428,-0.465519,-0.452091,-0.373863,...,-0.388959,-0.334955,-0.553681,-4.604247,-0.061377,-2.171497,5a0546857ecc773753327266,2.0,5a0546857ecc773753327266,2.0
4,-0.275976,-0.313008,-0.311924,-0.387360,-0.305805,0.402686,-0.339808,-0.337189,-0.307327,-0.196415,...,-0.305951,-0.159625,-0.291321,7.241242,-0.048594,3.717594,5a0546857ecc773753327266,3.0,5a0546857ecc773753327266,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,0.856125,0.709377,0.551538,0.884107,0.839942,0.362343,0.842541,0.753898,0.799924,0.470566,...,0.738607,0.522169,0.792181,6.999106,0.136614,3.395569,5dc8cea7659e181adb076a3f,2.0,5dc8cea7659e181adb076a3f,2.0
135,0.105272,0.027448,0.043425,0.108272,0.178791,-0.231798,0.098895,0.084849,-0.015210,-0.015718,...,0.178675,0.097601,0.105549,5.038488,0.013488,2.490602,5dc8cea7659e181adb076a3f,3.0,5dc8cea7659e181adb076a3f,3.0
136,0.566924,0.290470,0.334826,0.531453,0.557549,-0.123998,0.522505,0.483700,0.410564,0.248239,...,0.383466,0.388164,0.528496,1.381899,0.082125,0.560051,5dc8cea7659e181adb076a3f,4.0,5dc8cea7659e181adb076a3f,4.0
137,0.290740,0.391322,0.149305,0.279618,0.333365,0.513132,0.272129,0.280936,0.155004,0.089002,...,0.313316,0.231608,0.275085,1.792049,0.026478,0.786748,5dc8cea7659e181adb076a3f,5.0,5dc8cea7659e181adb076a3f,5.0


In [46]:
meta_emb_df.to_csv('./meta_emb_df.csv', index=False)